In [13]:
from transformers import GPT2LMHeadModel
from transformers import pipeline, set_seed, PretrainedConfig
import os
import torch
from torch.nn import functional as F
from train_gpt2 import *
import tiktoken

os.environ['XDG_CACHE_HOME'] = './cache'
os.environ['HF_HOME'] = './cache'
os.environ['HF_DATASETS_CACHE'] = './cache'
os.environ['TRANSFORMERS_CACHE'] = './cache'
os.environ['HUGGINGFACE_HUB_CACHE'] = './cache'


In [37]:
checkpoint = torch.load('/Users/germankochnev/Desktop/projects/chatgpt2/lg/model_19072.pt', map_location='cpu', weights_only=False)
tokenizer = tiktoken.get_encoding('gpt2')

In [38]:
weights = checkpoint['model']
weights = {k.replace('_orig_mod.', ''): v for k, v in weights.items()}

In [39]:
model = GPT(GPTConfig(vocab_size=50304))
model.load_state_dict(weights)
model.eval()

GPT(
  (transformer): ModuleDict(
    (wte): Embedding(50304, 768)
    (wpe): Embedding(1024, 768)
    (h): ModuleList(
      (0-11): 12 x Block(
        (ln_1): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
        (attn): CausalSelfAttention(
          (c_attn): Linear(in_features=768, out_features=2304, bias=True)
          (c_proj): Linear(in_features=768, out_features=768, bias=True)
        )
        (ln_2): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
        (mlp): MLP(
          (c_fc): Linear(in_features=768, out_features=3072, bias=True)
          (gelu): GELU(approximate='tanh')
          (c_proj): Linear(in_features=3072, out_features=768, bias=True)
        )
      )
    )
    (ln_f): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
  )
  (lm_head): Linear(in_features=768, out_features=50304, bias=False)
)

In [49]:
generation_length = 100
temperature = 1
number_of_sentences = 5


prompt = 'Hello! My name is'
prompt_encoded = torch.tensor(tokenizer.encode(prompt)).unsqueeze(0)
prompt_encoded = prompt_encoded.repeat(number_of_sentences, 1)


with torch.no_grad():
    for _ in range(generation_length):
        out = model(prompt_encoded)
        logits = out[0][:, -1, :] / temperature
        next_token = torch.multinomial(F.softmax(logits, dim=-1), num_samples=1)
        prompt_encoded = torch.cat([prompt_encoded, next_token], dim=-1)

In [50]:
tokenizer.decode(prompt_encoded[1].tolist())

'Hello! My name is Carolyn and there is no doubt you will notice more posts on near the top. Where I’m from because some posts I have had show through the oil pastels haven’t pressed so hard. We’ve had forums about swimming some ones have side videos of various science experiments on YouTube. Whenever we had side videos of some teachers posting new photos I would link everyone to what we have in the school.\nWith you hopefully, this also teaches what to look for when you find'